# Tutorial 5: Curve Trees (The "Select" Gadget)

### The Concept: Algebraic Selection
In a Merkle Tree, the verification path depends on whether you are the Left or Right child.
* If Left: `Hash(You, Sibling)`
* If Right: `Hash(Sibling, You)`

Revealing "Left" or "Right" destroys anonymity. We need a way to select the correct order mathematically, using a secret bit $b$.

$$ Selected = L + b \cdot (R - L) $$

1. If $b=0$ (Left): $L + 0 = L$
2. If $b=1$ (Right): $L + (R - L) = R$

This "Select Gadget" allows us to verify the tree structure without revealing our position.

In [ ]:
extern crate kn0syseccrs as ecc_rs;
extern crate num;

In [ ]:
use ecc_rs::{Point, Scalar, hash_to_scalar, G, EccError};
use num::BigInt;

// Helper: Create a scalar from a generic integer
fn int_to_scalar(i: u64) -> Scalar {
    Scalar::new(BigInt::from(i)).unwrap()
}

println!("Libraries loaded.");

### Step 1: Implement the Select Gadget

We need a function that takes two points (`Left`, `Right`) and a bit (`Scalar 0 or 1`), and returns the selected point.

In [ ]:
fn select_node(left: &Point, right: &Point, bit: &Scalar) -> Point {
    // 1. Calculate diff = (R - L)
    let diff = (right.clone() - left.clone()).unwrap();
    
    // 2. Calculate term = bit * diff
    // If bit is 0, term is 0. If bit is 1, term is diff.
    let term = (diff * bit.clone()).unwrap();
    
    // 3. Result = L + term
    (left.clone() + term).unwrap()
}

println!("Select gadget defined.");

### Step 2: Verify the Logic

Let's create two points, `A` and `B`, and prove that our math works like an `if/else` statement.

In [ ]:
// Setup: Create two distinct points A and B
let priv_a = int_to_scalar(100);
let point_a = (G.clone() * priv_a).unwrap();

let priv_b = int_to_scalar(200);
let point_b = (G.clone() * priv_b).unwrap();

// Create bits 0 and 1
let bit_zero = int_to_scalar(0);
let bit_one = int_to_scalar(1);

// Test 1: Select with 0 (Should return A)
let result_zero = select_node(&point_a, &point_b, &bit_zero);
println!("Selection 0 Matches A? {}", result_zero == point_a);

// Test 2: Select with 1 (Should return B)
let result_one = select_node(&point_a, &point_b, &bit_one);
println!("Selection 1 Matches B? {}", result_one == point_b);

// Test 3: Failure Case
// If we use a bit that isn't 0 or 1 (e.g., 2), the math breaks (returns 2B - A).
let bit_two = int_to_scalar(2);
let result_garbage = select_node(&point_a, &point_b, &bit_two);
println!("Selection 2 Matches B? {}", result_garbage == point_b);
println!("Selection 2 Matches A? {}", result_garbage == point_a);

### Step 3: Navigating a Tree Layer

Now we verify a single layer of a Curve Tree. 
The Prover provides a `Sibling` and a `Bit`. We are at `CurrentNode`.

We need to arrange them into `Left` and `Right` to hash them correctly.

$$ LeftChild = Select(Current, Sibling, Bit) $$
$$ RightChild = Select(Sibling, Current, Bit) $$

If $Bit=0$: Left=Current, Right=Sibling.
If $Bit=1$: Left=Sibling, Right=Current.

This effectively swaps them if the bit is 1.

In [ ]:
// Scenario: We are the Left Child (A). Sibling is B. Bit is 0.
let current = point_a.clone();
let sibling = point_b.clone();
let my_bit = bit_zero.clone();

// 1. Reconstruct the layer order
let left_child = select_node(&current, &sibling, &my_bit);
let right_child = select_node(&sibling, &current, &my_bit);

println!("Reconstructed Left:  {}", left_child.get_hex());
println!("Reconstructed Right: {}", right_child.get_hex());

println!("Is Left A? {}", left_child == point_a);
println!("Is Right B? {}", right_child == point_b);

// In a real Curve Tree, we would now hash these two children to find the parent.
// Parent = Hash(left_child, right_child)